# Hypothesis testing

This notebook will teach you how to perform simple statistical tests you can use to check your hypotheses. It shows you how to calculate the P-values for binomial tests, in which you determine the likelyhood that the result of a binary choice is merely due to chance, and how to perform T-tests, where you calculate how likely it is that there is some meaningful difference between two samples.


In [23]:
# Import libraries
import numpy as np
import scipy.stats as st


## Performing a binomial test

A binomial test tests whether the difference between two categories is due to chance or not. Often, the null hypothesis is that the two categories occur with equal frequency (0.5) or in other words, that differences between them are due to chance. If the difference in frequency between the two categories in our data is different, we would like to know how *confident* we can be that this is not just due to chance but instead due to some underlying difference between the two categories.

Here is an example:

A company sells two similar products P1 and P2. The company expects it will sell less of the more expensive product P1 (even though it has more features), because it costs 1.5 times as much as P1. The company therefore expects that of the total number of items of P1 and P2 sold, P1 will sell at a proportion of 40%, with 60% for P2. So far, 48 items of P1 were sold, and 95 of P2. Is the company correct in assuming that the sales proportion will be the same as the price proportion?

To find out whether this is the case we calculate the P-value for a binomial test where we assume the choice between products P1 and P2 is entirely random, so 50/50 and where the proportion between the number of products P1 and P2 sold is the same as the observed proportion. Larger P-values mean a larger probability that the observed difference is due to chance.

To do this, we use the `binomtest` function in the `scipy.stats` package. Relevant parameters for this function are:
- `k`: the number of "successes", in this case the number of P1 products sold.
- `n`: the number of "trials", in this case the total number of products sold.
- `alternative` : the "alternative", that is to say, null hypothesis. "two-sided" means: the choice between P1 and P2 is entirely random.



In [ ]:
num_p1_sold = 48
num_p2_sold = 95

print(f"Observed proportion is {num_p1_sold / (num_p1_sold + num_p2_sold) :.2f} ")
# Calculate P-value
btresult = st.binomtest(k = num_p1_sold, 
                        n = num_p1_sold + num_p2_sold, alternative = 'two-sided')
print(f'P-value is: {btresult.pvalue :.4f}')

Observed proportion is 0.34 
P-value is: 0.0001


Apparently it is very, very unlikely that a 0.34 split between sales of P1 and P2 is due to chance. It is perhaps safe to assume there is some difference between the two products that accounts for the observed difference. Whether that is the price or something else, we can of course not know.

It is interesting to check *how far* the proportion of P1 products sold is from the true random chance of 0.5. To do this, we can calculate the *confidence interval* for our observation, meaning the range of values we can expect our observation to fall in given a sensible variance for, in this case, a binomial distribution.

The `binomtest` calculates this confidence interval for us.

In [18]:
print(f'Confidence interval is {btresult.proportion_ci().low :.2f} - {btresult.proportion_ci().high :.2f}')

Confidence interval is 0.26 - 0.42


We see that random chance would make it possible for the number of sales of the P1 product to vary between 0.26 and 0.42 given the observed proportion. This range does not contain the proportion we would have if the difference in sales would purely be due to chance (0.5) so again, we conclude there is some sort of mechanism at play that causes more of product P2 to be sold than of product P1.

## T-tests

Sometimes we want to compare two samples to each other to see if there is a meaningful difference between the two. If both samples are normally distributed, we can use what is called a T-test (also known as Student's T-test). This test calculates how likely it is that the difference between the two samples is statistically significant or not.

There are two versions of this test: the *paired* T-test and the *unpaired* T-test
- The paired T-test is used to calculate the T-test for to related samples, meaning samples where each datapoint is in some way related to the corresponding datapoint in the other sample. An example would be two samples of the same patients, one before an intervention and one after.
- The unpaired T-test is used to calculate the T-test for two independent samples, meaning samples that are not otherwise related, say patients who received an intervention and those that did not.

Python's T-test functions return a P-value (the probability that the observed difference is due to chance) and a so-called T-statistic. You can look up the exact meaning of this T-statistic on the internet, but, essentially, values that are further removed from 0 are better. Normally we are only interested in the P-value.

Since T-tests make use of the fact that the samples are normally distributed, the only parameters they need are the mean, the standard deviation and the number of samples. At least, this is true in theory and for the `scipy.stats.ttest_ind` method for unpaired T-tests, this is indeed true.

Here is an example of an unpaired T-test. We want to know if, in a given web store, people from the North spend more than people from the South.

In [21]:
# Unpaired T-test
n_mean = 73.24
n_sd = 12.33
n_n = 12108

s_mean = 76.81
s_sd = 11.80
s_n = 14952

tstat, pvalue = st.ttest_ind_from_stats(n_mean, n_sd, n_n, s_mean, s_sd, s_n)
print(f'P-value for difference between means is {pvalue :.8f}')


P-value for difference between means is 0.00000000


The observed difference is statistically significant, so there is some meaningful difference between customers from the North and customers from the South.

Now to do a paired T-test. Unfortunately, `scipy.stats` requires the data itself to perform this test (the `ttest_ind_from_stats` is a recent addition).

Let's say we have a group of patients with a fever. We would like to know if their fever drops meaningfully after giving them some new medicine.

We need to do some housekeeping to generate this data, given a mean, a standard deviation and an n. But other than that, the process is the same.



In [34]:
num_patients = 134
before_mean = 38.1
before_sd = 1.2
after_mean = 37.8
after_sd = 1.6

# Generate the data points
before = np.random.normal(before_mean, before_sd, num_patients)
after = np.random.normal(after_mean, after_sd, num_patients)

tstat, pvalue = st.ttest_rel(before, after)
print(f'P-value for difference between means is {pvalue :.2f}')




P-value for difference between means is 0.03
